In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [23]:
data = pd.read_csv('./dataset_spam.csv')
data.tail()

,content,label
6663,f a n s b e t i n g agent bola terpercaya di i...,spam
6664,pasti ada campur tangan ulah orang dalam karen...,normal
6665,mauuu puunyaa kullittt puutiiihh secaaraa inns...,spam
6666,bagi aku kaaa dikit aja,normal
6667,mila sipitt allah tuh benci sama orang yg meru...,normal


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6668 entries, 0 to 6667
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  6668 non-null   object
 1   label    6668 non-null   object
dtypes: object(2)
memory usage: 104.3+ KB


In [26]:
data.nunique()

content    5802
label         2
dtype: int64

In [27]:
# X = data.content
# Y = data.label
# label_encoder = LabelEncoder()
# X = label_encoder.fit_transform(Y)
# Y = Y.reshape(-1,1)
X = data.content
Y = data.label
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

c:\Users\Hammam\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Hammam\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [28]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequence = tok.texts_to_sequences(X_train)
sequence_matrix = pad_sequences(sequence, maxlen=max_len)

In [29]:
sequence_matrix

array([[  0,   0,   0, ..., 325,   2,  36],
       [  0,   0,   0, ...,  69, 806, 949],
       [  0,   0,   0, ...,   2, 118,  20],
       ...,
       [  0,   0,   0, ..., 767, 240,   2],
       [  0,   0,   0, ..., 298, 605,  14],
       [  0,   0,   0, ..., 166, 797, 243]])

In [38]:
def RNN():
    inputs = Input(name='iputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer_s = Activation('sigmoid')(layer)
    layer_t = Activation('tanh')(layer)
    model = Model(inputs=inputs, outputs=[layer_s, layer_t])
    return model

In [39]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 iputs (InputLayer)             [(None, 150)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 150, 50)      50000       ['iputs[0][0]']                  
                                                                                                  
 lstm_2 (LSTM)                  (None, 64)           29440       ['embedding_2[0][0]']            
                                                                                                  
 FC1 (Dense)                    (None, 256)          16640       ['lstm_2[0][0]']                 
                                                                                            

In [31]:
model.fit(sequence_matrix, Y_train, batch_size=128, epochs=10, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
30/30 [==============================] - 48s 899ms/step - loss: 0.3584 - accuracy: 0.8615 - val_loss: 0.2440 - val_accuracy: 0.9379
Epoch 2/10
30/30 [==============================] - 23s 762ms/step - loss: 0.1993 - accuracy: 0.9424 - val_loss: 0.1766 - val_accuracy: 0.9497
Epoch 3/10
30/30 [==============================] - 19s 648ms/step - loss: 0.1468 - accuracy: 0.9553 - val_loss: 0.1860 - val_accuracy: 0.9507


In [32]:
test_sequence = tok.texts_to_sequences(X_test)
test_sequence_matrix = pad_sequences(test_sequence, maxlen=max_len)

In [35]:
unique, counts = np.unique(Y_train, return_counts=True)

In [36]:
counts

array([4009,  658], dtype=int64)

In [33]:
model.predict(test_sequence_matrix)

63/63 [==============================] - 13s 111ms/step


array([[9.9737185e-01],
       [1.2477693e-04],
       [4.7551682e-03],
       ...,
       [7.8424963e-04],
       [1.1881952e-03],
       [1.4189448e-03]], dtype=float32)

In [34]:
accr = model.evaluate(test_sequence_matrix, Y_test)

63/63 [==============================] - 6s 86ms/step - loss: 0.1867 - accuracy: 0.9450


In [37]:
print('Test set\nLoss: {:0.3f}\nAccuarcy: {:0.3f}'.format(accr[0], accr[1]))

Test set
Loss: 0.187
Accuarcy: 0.945
